# **Make Trade History**

<reference>
    [Markdown](https://datanetworkanalysis.github.io/2019/10/12/markdown)

## Install & Update Libraries

In [65]:
#!pip install --quiet --upgrade pip requests beautifulsoup4 pandas pd
#!pip install --quiet --upgrade pip requests beautifulsoup4 pandas pd matplotlib seaborn selenium

!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' | tee /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable

!google-chrome --version

!wget -N https://chromedriver.storage.googleapis.com/88.0.4324.27/chromedriver_linux64.zip

!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv -f chromedriver /usr/local/share/chromedriver
!ln -s /usr/local/share/chromedriver /usr/local/bin/chromedriver
!ln -s /usr/local/share/chromedriver /usr/bin/chromedriver
!rm chromedriver_linux64.zip
!apt-get update
!apt-get install -y libgconf-2-4

In [66]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import os
import selenium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline
requests.__version__, pd.__version__, np.__version__, sns.__version__

('2.25.1', '1.2.2', '1.19.4', '0.11.1')

In [67]:
import bs4
import matplotlib
bs4.__version__, matplotlib.__version__

('4.9.3', '3.3.4')

## get real fee data of NH

In [68]:
url = "https://www.nhqv.com/WMDoc.action?viewPage=/guestGuide/business/feeBuySell.jsp&ga_referrer=https%3A%2F%2Fwww.google.com%2F"
#url

In [69]:
html = requests.get(url)
html.raise_for_status()
#html.content

In [70]:
soup = BeautifulSoup(html.text, 'html.parser')
#soup

tbody = soup.select_one("#contents > div:nth-child(2) > table > tbody")
trs = tbody.find_all('tr')
columns_=["min","max","expression"]
tds = map(lambda x:x.find_all('td').text, trs)
td = map(lambda x:x.text,trs)
arr = np.array(td)
df = pd.DataFrame(columns=columns_).astpye({
    'min':'int32'
    'max':'int32'
    'expression':'object'
})
td = list(td)[:9]
for i, tset in enumerate(td):
    temp = tset.split('\n')
    textset = temp[1:len(temp)-1]
    money,expression=tuple(textset[2:]) if i == 0 else tuple(textset[0:2])
    money = money.replace("미만","").replace(" ", "")\
                    .replace('백','00').replace('천','000').replace('만','0000')\
                        .replace("억","00000000").replace("원","").replace("이상","")
    money = money.split("~")
    expression = expression.replace('%','')
    expression = expression.replace('원','')
    expression = '*' + expression
    if i == 0:
        df = df.append(pd.DataFrame([[np.nan]+money+[expression]],columns=columns_))
    elif i == len(td)-1:
        df = df.append(pd.DataFrame([money+[np.nan]+[expression]],columns=columns_))
    else:
        df = df.append(pd.DataFrame([money+[expression]],columns=columns_))

## get real time price by beautifulsoup
### problem: program execution speed

In [71]:
dt = datetime.today().strftime("%Y/%m/%d")
#dt

In [72]:
code = '051910'
url = f"https://finance.naver.com/item/main.nhn?code={code}"

html = requests.get(url)
html.raise_for_status()
#url

In [73]:
import datetime

stock_code = "005930"
        
url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"
html = requests.get(url)
html.raise_for_status()
    
content = BeautifulSoup(html.content, 'html.parser')
    
stock_div = content.find("div",{"class":"wrap_company"})
stock_name = stock_div.find("a").text

feed = content.find("p",{"class":"no_today"})
price = int(feed.find("span",{"class":"blind"}).text.replace(",",""))

#now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#KST = timezone('Asia/Seoul')
now = datetime.datetime.utcnow()
#now = KST.localize(now)
#now = now.strftime("%Y-%m-%d %H:%M:%S")
now = pd.to_datetime(now,format="%Y-%m-%d %H:%M:%S", errors='raise')
now_local = now.tz_localize('Asia/Seoul')

stock_data = pd.DataFrame([[now_local, stock_name, price]],
                          columns=["requested_time","name","price"]).astype({'price':"int32"})

#stock_data

## get real time price by  selenium (????)

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

code = '005930'
url_ = f"https://finance.daum.net/quotes/A{code}#home"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
#can't crawl real time data?
driver = webdriver.Chrome(executable_path='chromedriver', options=chrome_options)
driver.get(url=url_)

value = driver.find_elemment_by_css_selector("#boxSummary > div > span:nth-child(1) > span.currentB > span.numB.down > strong")
value.text


## make history

In [74]:
data = pd.DataFrame({'date':pd.date_range("2020-01-03",periods=3),'price':[5,4,3],'quantity':[4,11,2]}
                    ,columns=["date","price","quantity"]
                    ,index=list("abc"))
#data.loc[data.idxmax()["hm"]]
temp = data.loc["b"]
temp2 = data.loc["c"]
data.loc["c"] = temp
data.loc["b"] = temp2
data.loc[data["date"].idxmax()]["price"]

3

In [75]:
data = pd.DataFrame(np.arange(25).reshape((5,5)),columns=list("abcde"))
data["index"] = ["a","a","b","a", "b"]
data["index2"] = ["c","c","d","c","d"]
data.groupby(["index","index2"])[["c","d"]].sum()
data["money_used"] = data["b"]*data["c"]
#data


In [76]:
data = data.groupby(["index","index2"])[["c","d","money_used"]].sum()
data = data.reset_index().set_index("index")
#data

In [77]:
temp = pd.DataFrame(columns=list('abc'))
temp["d"] = [3]
#temp.dtypes

In [78]:
import datetime
from pytz import timezone, utc

KST = timezone('Asia/Seoul')
now = datetime.datetime.utcnow()
#utc.localize(now)
#KST.localize(now)
now_local = utc.localize(now).astimezone(KST)

temp_date = datetime.datetime(2021,2,2,1,0,15,881617)
temp_date = utc.localize(temp_date).astimezone(timezone("Asia/Seoul"))

#temp_date > now_local

In [79]:
history_file_name = "../data/history/"+"purchase_history_" + "1" + ".csv"
#fd = open(history_file_name,'w')
#fd.close()

dt_parser = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S+%Z")
history = pd.read_csv(history_file_name,
                      names=["wow","jeon","yeok"]).astype({"wow":"category"})
history["yeok"]=pd.to_datetime(history["yeok"],
                               format="%Y-%m-%d %H:%M:%S+%z",
                               errors="raise")
len(history)


2

In [80]:
a = pd.DataFrame(np.arange(15).reshape((5,3)),columns=list("abc"))
abs(-4)

4

In [81]:
#import useful_function as uf
#import crawler
import os
import pytz

def isFileExist(file_name):
    """
    Check if file( or directory) exists
    if file exists, it returns True.
    """
    return os.path.exists(file_name)

def getRecentFeeData(path="../data/fee/"):
    if isFileExist(path):
        file_list = os.listdir(path)
        if len(file_list) > 0:
            file_name = max(file_list)
            data = pd.read_csv(path+file_name)
            return data
        else:
            raise Exception("No Fee Data File : " + path)
    else:
        raise Exception("Directory not Found : " + path)
        
def getNet(trade_type, price, tax = 0.003):
    fee_data = getRecentFeeData()
    net = 0
    price = abs(price)
    
    min_df = fee_data[fee_data["min"] <= price]
    max_df = fee_data[fee_data["max"] > price]
    inner_row = pd.concat([min_df,max_df],axis=1,join="inner").T.drop_duplicates()
    fee_expression = inner_row.loc["expression",inner_row.columns[0]]
    net += int(eval(str(price)+fee_expression))
    if not trade_type:
        net += int(price * tax)
    return net
    
        
def checkStockCode(stock_code):
    """ Check whether the stock code exists
    
    Parameters
    ----------
    stock_code : str
        the stock code (ex. "005930")
    """
    url = f"https://finance.daum.net/quotes/A{stock_code}#home"
    
    html = requests.get(url)
    html.raise_for_status()
    
    return "alert" not in html.text

def getCurrentStockData(stock_code, timezone="Asia/Seoul"):
    """
    Return current data of requested company
    Parameters
    ----------
    stock_code : str
        the stock code (ex. "005930")
    Return Value
    ----------
    DataFrame
    ex)
    requested_time	                 name    price
    2021-01-30 05:31:34.130440+09:00 삼성전자 82000
    """
    now_local = getLocalTime(timezone)
    
    if not checkStockCode(stock_code):
        raise ValueError("Nonexistent stock code")
    url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"
    html = requests.get(url)
    html.raise_for_status()

    content = BeautifulSoup(html.content, 'html.parser')
    
    stock_div = content.find("div",{"class":"wrap_company"})
    stock_name = stock_div.find("a").text

    feed = content.find("p",{"class":"no_today"})
    price = int(feed.find("span",{"class":"blind"}).text.replace(",",""))
    
    stock_data = pd.DataFrame([[now_local, stock_name, price]],
                          columns=["requested_time","stock_name","price"]).astype({
                            "stock_name":"object",
                            "price":"int32"})
    
    return stock_data

def getLocalTime(area = "Asia/Seoul"):
    """
    Return local time whose format is "Year-Month-Day Hour:Minute:Second"
    Default area is seoul.
    UTC clock(?) is based on UK, so we need to localize clock.
    """
    KST = pytz.timezone(area)
    now = datetime.datetime.now(tz=pytz.timezone(area))
    #now_local = pytz.utc.localize(now).astimezone(KST)
    return now

def checkMarketOpened(now, area="Asia/Seoul"):
    #limited for Korea market
    date = datetime.datetime.today().date()
    open_time = datetime.datetime.combine(date,datetime.datetime(2021,1,1,9,0).time(),
                                         tzinfo=pytz.timezone("Asia/Seoul"))
    close_time = datetime.datetime.combine(date,datetime.datetime(2021,1,1,15,30).time(),
                                         tzinfo=pytz.timezone("Asia/Seoul"))

    return True if now > open_time and now < close_time else False

getNet(False, 1000000)

5441

In [82]:
def getNet(trade_type, price, tax = 0.003):
    fee_data = getRecentFeeData()
    net = 0
    price = abs(price)
    
    min_df = fee_data[fee_data["min"] <= price]
    max_df = fee_data[fee_data["max"] > price]
    inner_row = pd.concat([min_df,max_df],axis=1,join="inner").T.drop_duplicates()
    fee_expression = inner_row.loc["expression",inner_row.columns[0]]
    net += int(eval(str(price)+fee_expression))
    if not trade_type:
        net += int(price * tax)
    return net

getNet(False,100500)

802

In [83]:
stock_data = pd.DataFrame([[1,"005930",4,30],[2,"123456",5,50],
                  [3,"123523",5,40],[4,"213523",np.nan,66]],columns=list("abcd"),
                          index=["삼성전자","작통단","휴가","한화생명"])
#stock_data

In [84]:
new_stock_data = pd.DataFrame([[5,"262432",5,222]],columns=list("abcd"),index=["wow"])
#stock_data = pd.concat([stock_data,new_stock_data])
new_stock_data

,a,b,c,d
wow,5,262432,5,222


In [85]:
stock_data = pd.concat([stock_data,new_stock_data])
#stock_data

In [86]:
now_local = getLocalTime()
balance = 200231
c = ["date","stock_code","stock_name","price","quantity","total_amount","balance"]
d = [now_local,np.nan,np.nan,np.nan,np.nan,np.nan,balance]

temp = pd.DataFrame([d],columns=c)
temp.loc[temp.index[0],"stock_code":"total_amount"].isnull().all()

True

In [87]:
date = datetime.datetime.today().date()
open_time = datetime.datetime.combine(date,datetime.datetime(2021,1,1,0,0).time(),
                                         tzinfo=pytz.timezone("Asia/Seoul"))
close_time = datetime.datetime.combine(date,datetime.datetime(2021,1,1,6,30).time(),
                                         tzinfo=pytz.timezone("Asia/Seoul"))
pd.DataFrame([[getLocalTime()]]).dtypes

0    datetime64[ns, Asia/Seoul]
dtype: object

In [88]:
temp1 = pd.DataFrame([[getLocalTime(),"005930","삼성전자",79000,10,790000,444444]],
                    columns=["date","stock_code","stock_name",
                             "price","quantity","total_amount","balance"]).astype({
                                                "stock_code":"object",
                "stock_name":"object",
                "price":"int64",
                "quantity":"int64",
                "total_amount":"int64",
                "balance":"int64"})
                            
temp2 = pd.DataFrame([[getLocalTime(),"005922","한화",790,4,3160,444444]],
                    columns=["date","stock_code","stock_name",
                             "price","quantity","total_amount","balance"]).astype({
                                                "stock_code":"object",
                "stock_name":"object",
                "price":"int64",
                "quantity":"int64",
                "total_amount":"int64",
                "balance":"int64"})
temp1.shape[0]

1

In [89]:
time_type = pd.Series([getLocalTime()]).dtype
wo = [time_type] + ["object","object","int64","int64","int64","int64"]
#wo

In [90]:
import os
import re
#히스토리 쓰는 클래스
#쓰기, 상태 리턴(가지고 있는 주식 개수), 
#csv파일 명은 purchase_history_[strategy_index]
#시간 관련 함수 만들어야 할 듯 utc + 9
#타입 제대로 적기
#단 history와 trade간 업데이트 하는 함수가 하나 필요할 듯 -> 진위 판단
#history에서 거래에 대한 요약본을 관리하는 대신 트레이드와 히스토리 클래스간 업데이트하는 함수 만들자

def getRecentFeeData(path="../data/fee/"):
    if isFileExist(path):
        file_list = os.listdir(path)
        if len(file_list) > 0:
            file_name = max(file_list)
            data = pd.read_csv(path+file_name)
            return data
        else:
            raise FileNotFoundError("No Fee Data File : " + path)
    else:
        raise FileNotFoundError("Directory not Found : " + path)
        
def checkMarketOpened(now, area="Asia/Seoul"):
    #limited for Korea market
    date = datetime.datetime.today().date()
    open_time = datetime.datetime.combine(date,datetime.datetime(2021,1,1,0,0).time(),
                                         tzinfo=pytz.timezone("Asia/Seoul"))
    close_time = datetime.datetime.combine(date,datetime.datetime(2021,1,1,6,30).time(),
                                         tzinfo=pytz.timezone("Asia/Seoul"))
    
    return True if now > open_time and now < close_time else False
    
#조건 체크
#shape
#마켓열려있는지도 확인
class History:
    def __init__(self, strategy_id, seed_money = 1000000, data_path="../data/history/"):
        self.data_path = data_path
        self.strategy_id = strategy_id
        self.history_file_name = data_path + "purchase_history_" + str(self.strategy_id) + ".csv"
        self.columns = ["date","stock_code","stock_name","price","quantity","total_amount","net","balance"]
        self.dtypes = ["object","object","int64","int64","int64","int64","int64"]
        self.fd = open(self.history_file_name,"a").close()
        self.balance = 0
        self.stock_data, self.balance, self.recent_trade_date = self.setStatus(seed_money)
        
    def readHistoryFile(self):
        if isFileExist(self.history_file_name):
            try:
                #format="%Y-%m-%d %H:%M:%S+%z"
                history = pd.read_csv(self.history_file_name,
                            dtype={"stock_code":"object","stock_name":"object"},
                            names=self.columns).astype({"stock_code":"category","stock_name":"category"})
                history["date"]=pd.to_datetime(history["date"],errors="raise")
                return history
            except pd.errors.EmptyDataError:
                raise pd.errors.EmptyDataError("No Data to Read : " + history_file_name)
        else:
            raise FileNotFoundError("No History File : " + history_file_name)
    #데이터 확인
    def writeHistoryFile(self, data):
        if not isFileExist(self.history_file_name):
            raise FilenotFoundError("No History File : " + history_file_name)
        #date_format="%Y-%m-%d %H:%M:%S+%Z"
        data.to_csv(self.history_file_name,mode='a',header=False,index=False)

    def setStatus(self, seed_money):
        #현재 가지고 있는 전체 주식들 정보 / 현재 남은 돈
        #입금한건 어떻게 읽누?
        #stock_data dataframe
        #stock_code | stock_name quantity money_used
        
        history = self.readHistoryFile()
        if history.shape[0] > 0: #no data to read -> 초기 datagram 형성 불가
            #null data is not contained in groupby data
            stock_data = history.groupby(["stock_code","stock_name"])[["quantity","total_amount"]].sum()
            stock_data = stock_data.reset_index().set_index("stock_code")
            recent = history.loc[history["date"].idxmax()]
            recent_balance = recent["balance"]
            recent_date = recent["date"]
        else:
            self.writeHistoryFile(self.putMoney(seed_money))
            stock_data = pd.DataFrame(columns=["stock_code","stock_name","quantity","total_amount"])
            stock_data.set_index("stock_code")
            recent_balance = seed_money
            recent_date = getLocalTime()
        stock_data.astype({
            "stock_name":"object",
            "quantity":"int64",
            "total_amount":"int64"
            })
        return stock_data, recent_balance, recent_date  #입금액 = recent["balance"] - stock_data.sum()
    
    def getStatus(self, stock_code):
        try:
            if not re.compile("\d{6}").match(stock_code) and len(stock_code) != 6:
                raise Exception("Wrong Input : " + stock_code)
        except TypeError:
            raise TypeError("Wrong Input : " + stock_code)
        
        try:
            temp = self.stock_data.loc[stock_code]
            return temp
        except KeyError:
            raise KeyError("No Data : " + stock_code)
            
    def check_data(self,data):
        #column / dtype
        error_count = 0
        index = data.index[0]
        stock_code = data.loc[index,"stock_code"]
        is_column_same = True
        comp = pd.DataFrame([[getLocalTime(),"000000","0",0,0,0,0,0]],
                            columns=self.columns).astype(dict(zip(columns[self.columns.index("stock_code"):],self.dtypes)))
        
        if data.loc[index,"date"] < self.recent_trade_date:
            raise Exception("Time Error <Recent trade is occured earlier than now : "
                            + data.loc[index,"date"] + ">")
            
        if not (data.columns == comp.columns).all():
            error_count += 1
            is_column_same = False
            
        if data.loc[data.index[0],"stock_code":"total_amount"].isnull().all() and is_column_same: #입금 데이터
            if not data.dtypes.date == comp.dtypes.date or not data.dtypes.balance == comp.dtypes.balance:
                error_count += 1
                
        elif not (data.dtypes == comp.dtypes).all() and is_column_same:
            error_count += 1
            
        elif is_column_same:
            if not re.compile("\d{6}").match(stock_code) and not len(stock_code) == 6:
                error_count += 1
            if not data.loc[index,"price"] * data.loc[index,"quantity"] == data.loc[index,"total_amount"]:
                error_count += 1
        
        return False if error_count > 0 else True
    
    def isBuyable(self, data):
        #datagram에 문제 있으면 raise
        #주식 장이 안열려있으면 못함
        #잔고
        #최근 구매가 현재 시간을 넘으면 안됨 -> 이거는 함수로 하나 만들자
        #데이터 적절성 검사 코드 -> 모든 data 들어가는 함수에 쓰자 / 시간 검사 코드 / 마켓 열린지 확인
        error_count = 0
        if not self.check_data(data):
            error_count += 1
        else:
            if data.loc[data.index[0],"date_time"] < self.recent_trade_date:
                raise Exception("Time Error <Recent trade is occured earlier than now : "
                    + stock_data["requested time"] + ">")
            
            if self.balance - data.loc[data.index[0],"total_amount"] < 0:
                error_count += 1
        
        return True if error_count > 0 else False
        
    def isSellable():
        pass
        #최근 구매가 현재 시간을 넘으면 안됨
        #가지고 있는 주식개수가 살려는 주식 개수 넘으면 안됨
        
    #buyStock이랑 sellStock 합쳐서 그냥 dataframe 만드는 함수로 바꾸기
    def tradeStock(self, trade_type, stock_code, quantity):
        #trade_type = True -> 매수, False -> 매도
        #데이터 구성하는 함수
        ''' 
        https://spoqa.github.io/2012/08/03/about-python-coding-convention.html
        전반적인 함수명 조정이 필요할 듯 / column list 인스턴스로 올리기
        *quantity에서 판거는 -x 사는거는 x -> 파는 것과 사는것의 공식이 같다
        *현재 파일에는 column 저장하지 않음
        *현재 시간은 datetime64로 저장되있음. 다른 timezone끼리는 계산이 안됨. 해외 주식 할려면 고쳐야함
        한 파일에 여러 timezone이 있으면 to_datetime써도 object로 인식됨
        -> 다른 timezone은 다른 type의 object로 인식되는 듯
        -> 다른 timezone도 그냥 같은 utc clock으로 인식하게끔 만들어보기
        -> 다른 나라 주식은 하게되면 바꾸자..
        주식을 사고 팔 때 쓰는 함수
        주식 거래 -> write history -> history 인스턴스 업데이트
        '''
        try:
            if not re.compile("\d{6}").match(stock_code) and len(stock_code) != 6:
                raise Exception("Wrong Input : " + stock_code)
        except TypeError:
            raise TypeError("Wrong Input : " + stock_code)
            
        quantity = int(quantity) if trade_type else -int(quantity)
        if quantity == 0:
            raise Exception("Wrong Input : " + quantity)
            
        if not checkStockCode(stock_code):
            raise ValueError("Nonexistent stock code")
            
        stock_data = getCurrentStockData(stock_code).loc[0] #"requested_time","name","price"
        
        if stock_data["requested_time"] < self.recent_trade_date:
            raise Exception("Time Error <Recent trade is occured earlier than now : "
                            + stock_data["requested time"] + ">")
            
        total_amount = quantity * stock_data['price']
        net = getNet(trade_type, total_amount)
        balance = self.balance - total_amount - net #not update balance and stock_data yet
        
        info = [stock_data["requested_time"],stock_code,stock_data["stock_name"],
                   stock_data["price"], quantity, abs(total_amount), net, balance]
        
        data = pd.DataFrame([info],columns=self.columns).astype(dict(zip(self.columns[self.columns.index("stock_code"):],self.dtypes)))
        return data
            
    def putMoney(self, money):
        money = int(money)
        if money <= 0:
            raise Exception("Wrong Input : " + money)
        
        now_local = getLocalTime()
        balance = self.balance + money
        
        info = [now_local] + [np.nan]*6 + [balance]
        data = pd.DataFrame([info],columns=self.columns)
        
        return data
        
    def __update(self, data):
        #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
        if not len(data.index) == 1:
            raise Exception("Wrong Input : " + data)
        
        #stock_code | stock_name quantity money_used
        index_name = data.index[0]
        stock_code = data.loc[index_name,"stock_code"]
        
        if stock_code in self.stock_data.index:
            new_quantity = self.stock_data.loc[stock_code,"quantity"]+data.loc[index_name,"quantity"]
            self.stock_data.loc[stock_code,"quantity"] = new_quantity
            new_money_used = self.stock_data.loc[stock_code,"money_used"]+data.loc[index_name,"total_amount"]
            self.stock_data.loc[stock_code,"money_used"] = new_money_used
        
        else:
            #여기부터 새로운 정보일 경우 추가하는 부분
            info = data.loc[index_name,["stock_name","quantity","total_amount"]]
            new_row = pd.DataFrame([info],
                                   index=[data.loc[index_name,"stock_code"]],
                                   columns=["stock_name","quantity","total_amount"])
            self.stock_data = pd.concat([self.stock_data,new_row])
    
        self.balance = data.loc[index_name,"balance"]
        self.recent_trade_date = data.loc[index_name,"date"] 
    
    def update(self, data):
        self.__update(data)
    

In [91]:
h = History(77)

In [92]:
w = h.tradeStock(True , "005930",10)
w

,date,stock_code,stock_name,price,quantity,total_amount,net,balance
0,2021-02-15 17:13:15.472946+09:00,005930,삼성전자,84200,10,842000,2166,-662287


In [ ]:
w = h.tradeStock(False , "005930",10)
w

In [ ]:
columns = ["date","stock_code","stock_name","price","quantity","total_amount","net","balance"]
dtypes = ["object","object","int64","int64","int64","int64","int64"]
#dict(zip(columns[columns.index("stock_code"):],dtypes))

In [ ]:
import datetime

def getLocalTime(area = "Asia/Seoul"):
    """
    Return local time whose format is "Year-Month-Day Hour:Minute:Second"
    Default area is seoul.
    UTC clock(?) is based on UK, so we need to localize clock.
    """
    KST = pytz.timezone(area)
    now = datetime.datetime.now(tz=pytz.timezone(area))
    #now_local = pytz.utc.localize(now).astimezone(KST)
    return now

def checkMarketOpened(now, area="Asia/Seoul"):
    #limited for Korea market
    day_list = ['mon',"tue","wed","thu","fri","sat","sun"]
    day = day_list[now.weekday()]
    date = datetime.datetime.today().date()
    open_time = datetime.datetime.combine(date,datetime.datetime(2021,1,1,9,0).time(),
                                         tzinfo=pytz.timezone("Asia/Seoul"))
    close_time = datetime.datetime.combine(date,datetime.datetime(2021,1,1,15,30).time(),
                                         tzinfo=pytz.timezone("Asia/Seoul"))
    
    return True if (now > open_time and now < close_time) and day not in ["sat","sun"] else False

d = datetime.datetime(2021,2,10,10,10,23,180470)
d_local = pytz.utc.localize(d).astimezone(pytz.timezone("Asia/Seoul"))
day_list = ['mon',"tue","wed","thu","fri","sat","sun"]
a = day_list[d_local.weekday()]
checkMarketOpened(getLocalTime())


In [ ]:
import pandas as pd 
class WrongInputError(Exception):
    def __init__(self, value):
        self.value = str(value)
    def __str__(self):
        return "Wrong Input : " + self.value

class TimeError(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return "Time Error <Recent trade is occured earlier than now : " + self.value + ">"

    
raise TimeError(str(getLocalTime()))